In [2]:
!pip3 install great_expectations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 41.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: altair
    Found existing installation: altair 5.5.0
    Uninstalling altair-5.5.0:
      Successfully uninstalled altair-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of th

In [3]:
import great_expectations as gx
import pandas as pd

In [13]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/great-expectations/gx_tutorials/main/data/yellow_tripdata_sample_2019-01.csv"
)

In [14]:
df.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-15 03:36:12,2019-01-15 03:42:19,1,1.0,1,N,230,48,1,6.5,0.5,0.5,1.95,0.0,0.3,9.75,NaN
1,1,2019-01-25 18:20:32,2019-01-25 18:26:55,1,0.8,1,N,112,112,1,6.0,1.0,0.5,1.55,0.0,0.3,9.35,0.0
2,1,2019-01-05 06:47:31,2019-01-05 06:52:19,1,1.1,1,N,107,4,2,6.0,0.0,0.5,0.00,0.0,0.3,6.80,NaN
3,1,2019-01-09 15:08:02,2019-01-09 15:20:17,1,2.5,1,N,143,158,1,11.0,0.0,0.5,3.00,0.0,0.3,14.80,NaN
4,1,2019-01-25 18:49:51,2019-01-25 18:56:44,1,0.8,1,N,246,90,1,6.5,1.0,0.5,1.65,0.0,0.3,9.95,0.0


# Using Great Expectations

In [15]:
## Example Setup

context = gx.get_context()
data_source = context.data_sources.add_pandas("pandas")
data_asset = data_source.add_dataframe_asset(name="pd dataframe asset")

batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")
batch = batch_definition.get_batch(batch_parameters={"dataframe": df})

expectation = gx.expectations.ExpectColumnValuesToBeBetween(
    column="passenger_count", min_value=1, max_value=6
)

validation_result = batch.validate(expectation)

INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmpkgf0h3c7' for ephemeral docs site


Calculating Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

In [16]:
validation_result

{
  "success": true,
  "expectation_config": {
    "type": "expect_column_values_to_be_between",
    "kwargs": {
      "batch_id": "pandas-pd dataframe asset",
      "column": "passenger_count",
      "min_value": 1.0,
      "max_value": 6.0
    },
    "meta": {}
  },
  "result": {
    "element_count": 10000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_counts": [],
    "partial_unexpected_index_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


#**To-do:**

1.   Fetch any dataset from online source. I'll recommend using reddit API used in the assignment #1.
2.   Run 5 expectations of your choice to validate the dataset. They should cover row, column, multi-column, table, distribution operation.
3.   Submit the notebook file on LMS before Monday 11:55 PM.
4.   Mention your group number in the name of the file.



In [7]:
%pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.1 MB/s eta 0:00:00


In [8]:
from IPython.display import display
from datetime import datetime

import shutil
import praw
import time
import os

In [17]:
def get_posts_for_keyword(keyword, limit):
    # https://github.com/reddit-archive/reddit/wiki/OAuth2-Quick-Start-Example
    reddit = praw.Reddit(
        client_id="fzlFUxAw5BjWmNTnrmCCHg",
        client_secret="FiOYCbC0KqliZY6jOWCAZM1PPCZ_OA",
        user_agent="ai601_assignment_1"
    )

    posts = reddit.subreddit('all').search(keyword, limit=limit)
    # Lambda Function equivalent to inline/anonymous function in other languages
    toMap = lambda post: {
        'title': post.title,
        'text': post.selftext,
        'author': str(post.author),
        'date': datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
        'upvotes': post.score,
        'comments': post.num_comments
    }

    return [toMap(post) for post in posts]
def get_reddit_data(keywords):
    if (os.path.exists('./datasets/raw/reddit_posts.csv')):
        print("Reddit remote work data dataframe read from ./datasets/raw/reddit_posts.csv")
        return pd.read_csv('./datasets/raw/reddit_posts.csv', index_col=0)

    # Download the dataset from Reddit

    # List to store all posts
    posts_data = []

    # Search through each keyword
    for keyword in keywords:
        # 200 / 2 = 100 # 100 posts for each keyword
        number_of_posts_for_keyword = 200 // len(keywords)
        # Search for posts
        try:
            posts = get_posts_for_keyword(keyword, number_of_posts_for_keyword)
            # Extend -> addAll posts to the list
            posts_data.extend(posts)
            time.sleep(0.5) # To Prevent Rate Limiting 229 status code
        except Exception as e:
            print(f"Error processing posts for keyword {keyword}: {e}")
            raise e

    # Convert to DataFrame
    df = pd.DataFrame(posts_data)

    # Remove duplicates if any
    df = df.drop_duplicates(subset=['title', 'author', 'date'])

    return df

In [18]:
redditKeywords = ['RemoteWork','WorkFromHome']
reddit_df = get_reddit_data(redditKeywords)
print ('len', len(df))
display(reddit_df.head())

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



len 10000


,title,text,author,date,upvotes,comments
0,Is there any REAL full time work from home jobs?,Or are all those a big bunch of bullshit? I ha...,TangeloOdd9427,2025-01-02 18:39:52,2,39
1,Does anyone know of any legit work from home j...,My husband is the only one working right now d...,Sea_Panic9863,2022-03-09 20:56:26,323,289
2,Remote/Work from home jobs. What do you do?,I've been a salon owner/stylist for the last 2...,Icy_Nose_7413,2024-10-10 13:59:37,19,48
3,Are there any REAL work from home jobs anymore?,"Due to a family member that needs supervision,...",funkohunter717,2024-11-04 09:18:32,24,73
4,Work from home- remote jobs,I am currently awaiting the approval for my di...,Upbeat_Section8673,2024-08-08 23:53:09,69,93


In [24]:
context = gx.get_context()

data_source = context.data_sources.add_pandas("pandas")
data_asset = data_source.add_dataframe_asset(name="reddit_dataframe")

batch_definition = data_asset.add_batch_definition_whole_dataframe("batch_definition")
batch_request = batch_definition.build_batch_request(batch_parameters={"dataframe": reddit_df})

validator = context.get_validator(batch_request=batch_request)

expectations = [
    validator.expect_table_row_count_to_be_between(min_value=101, max_value=None),  # Table-level expectation
    validator.expect_column_values_to_not_be_null(column="title"),  # Column-level expectation
    validator.expect_column_values_to_be_in_set(column="author", value_set=["Sea_Panic9863", "funkohunter717", "random"]),  # Multi-column expectation
    validator.expect_column_mean_to_be_between(column="upvotes", min_value=10, max_value=10000),  # Distribution expectation
    validator.expect_column_pair_values_to_be_in_set(
        column_A="upvotes", column_B="comments", value_pairs_set=[(100, 10), (200, 20)]
    ),  # Row-level expectation
]

for result in expectations:
    print(result)

INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmp_wvza12t' for ephemeral docs site
/usr/local/lib/python3.11/dist-packages/great_expectations/expectations/expectation.py:1453: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/great_expectations/expectations/expectation.py:1453: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/great_expectations/expectations/expectation.py:1453: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(


Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/great_expectations/expectations/expectation.py:1453: UserWarning: `result_format` configured at the Validator-level will not be persisted. Please add the configuration to your Checkpoint config or checkpoint_run() method instead.
  warnings.warn(


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": false,
  "expectation_config": {
    "type": "expect_table_row_count_to_be_between",
    "kwargs": {
      "batch_id": "pandas-reddit_dataframe",
      "min_value": 101
    },
    "meta": {}
  },
  "result": {
    "observed_value": 100
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}
{
  "success": true,
  "expectation_config": {
    "type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "batch_id": "pandas-reddit_dataframe",
      "column": "title"
    },
    "meta": {}
  },
  "result": {
    "element_count": 100,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}
{
  "success": false,
  "expectation_config": {
    "type": "expect_column_values_to_be_in_set",
    "kwargs": {
      "batch